In [ ]:
from telethon import TelegramClient
from datetime import timedelta, datetime
import pytz
import pandas as pd
from telethon import functions, types
from tqdm import tqdm

In [ ]:
# Insert your information from https://my.telegram.org/
api_id = YOUR-API-ID #Integer
api_hash = 'YOUR-API-HASH' #String
phone = "YOUR-PHONE-NUMBER" #String

client = TelegramClient("anon", api_id=api_id, api_hash=api_hash)

await client.start(phone=phone)

In [ ]:
# Insert the required start date and the channel url, you'd like to retrieve
utc=pytz.UTC

start_date = "YYYY-MM-DD, HH:MM:SS"
start_date = datetime.strptime(start_date, "%Y-%m-%d, %H:%M:%S").replace(tzinfo=utc)

end_date = "YYYY-MM-DD, HH:MM:SS"
end_date = datetime.strptime(end_date, "%Y-%m-%d, %H:%M:%S").replace(tzinfo=utc)

channel_list = ["URL1",
                "URL2"] # Add the desired links to the Telegram channels as string to the list

In [ ]:
message_list = []

for url in channel_list:
    channel = await client.get_entity(url)

    current_date = start_date

    async for message in client.iter_messages(channel, limit=1, offset_date=end_date):
        latest_date = message.date

    while current_date < latest_date:
        async for message in client.iter_messages(channel, limit=None, offset_date=current_date, reverse=True):
            if message.date <= end_date:
                message_id = message.id
                date = message.date
                channel_id = message.peer_id.channel_id
                author = message.sender.title
                user_name = message.sender.username
                text = message.message
                views = message.views
                forwards = message.forwards
                try:
                    replies = message.replies.replies
                except AttributeError:
                    replies = None
                print(date, author)
                message_list.append([message_id, date, channel_id, author, user_name, text, views, forwards, replies])
        current_date = message.date

message_df = pd.DataFrame(message_list, columns=["message_id", "date", "channel_id", "author", "user_name", "text", "views", "forwards", "replies"])


In [ ]:
message_df.csv(f"{datetime.today().date()}_messages.csv")